In [1]:
using Interact
using CoordinateTransformations
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using ProfileView

In [2]:
import Flash
reload("Flash")

adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [3]:
model = Flash.Models.beanbag()
state = Flash.ManipulatorState(model)
Flash.draw(state);

In [4]:
geometry = collect(values(model.surfaces[1].geometries))[1]
@manipulate for x = linspace(-1, 1, 51)
    state.deformations[geometry][1][:] = [x, 0, 0]
    Flash.draw(state)
end;

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),".3f",true)

In [5]:
# Construct the sensor and draw its view rays for debugging
sensor = Flash.DepthSensors.Kinect(80,80);
camera_origin = [0., 0, 8]
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi, 1, 0, 0)))
Flash.DepthSensors.draw_rays(sensor, camera_tform)

In [6]:
true_state = Flash.ManipulatorState(model)
true_state.mechanism_state.q[5:7] = 2 * rand(3)
true_state.deformation_data[:] = 0.5 .* (rand(Float64, length(true_state.deformation_data)) .- 0.5)
Flash.draw(true_state);

In [7]:
# Now we can raycast that true state to get a set of sensed points:
sensed_points = Flash.DepthSensors.raycast(true_state, sensor, camera_tform)
Flash.DepthSensors.draw_points(sensed_points)

In [8]:
function test_gradient_descent(model, sensed_points, true_state, loops)
    x_true = copy(Flash.flatten(true_state))
    trials = Vector{DataFrame}()
    solver = NaiveSolver(length(x_true); rate=1.0, max_step=0.1, iteration_limit=2000)
    
    for i = 1:loops
        xs = Vector{Float64}[]
        costs = Float64[]
        
        function callback{T}(x::AbstractVector{T}, c)
            x_value = Flash.value.(x)
            state = Flash.ManipulatorState(model)
            Flash.GradientDescent.unflatten!(state, x_value)
            Flash.draw(state, false)
            push!(xs, x_value)
            push!(costs, Flash.value(c))
        end
        
        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        Flash.Tracking.estimate_state(model, sensed_points, x_estimated; callback=callback, solver=solver)
        push!(trials, DataFrame(x=xs, cost=costs))
        
        state = Flash.ManipulatorState(model)
        Flash.GradientDescent.unflatten!(state, xs[end])
        Flash.draw(state, true)
    end
    
    trials
end


test_gradient_descent (generic function with 1 method)

In [9]:
test_gradient_descent(model, sensed_points, true_state, 1);

In [10]:
Profile.clear()
@time @profile test_gradient_descent(model, sensed_points, true_state, 1);
# ProfileView.view()

  0.262801 seconds (1.70 M allocations: 109.393 MB, 8.93% gc time)


1-element Array{DataFrames.DataFrame,1}:
 276×2 DataFrames.DataFrame
│ Row │ ├─────┼
│ 1   │ │ 2   │ │ 3   │ │ 4   │ │ 5   │ │ 6   │ │ 7   │ │ 8   │ │ 9   │ │ 10  │ │ 11  │ 
⋮
│ 265 │ │ 266 │ │ 267 │ │ 268 │ │ 269 │ │ 270 │ │ 271 │ │ 272 │ │ 273 │ │ 274 │ │ 275 │ │ 276 │ 

│ Row │ x                                                                                                                                                                                                                                                                           │
├─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 1   │ [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]                                                                             